In [1]:
currReporterTrust= [0.25,0.71,0.79,0.87,0.95,0.21,0.59,0.72,0.89,0.68,0.37,0.47,0.27,0.27,0.71,0.99,0.68,0.59,0.35,0.61,0.56,.79,0.69,
0.79,0.04,0.96,0.43,0.77,0.99,0.73,0.91,0.03,0.25,0.72,0.77,0.23,0.33,0.43,0.47,0.06,0.76,0.19,0.97,0.23,0.38,0.28,0.46,0.30,0.31]

In [2]:
from ecdsa import SigningKey
sk = SigningKey.generate()
vk = sk.verifying_key
with open("private.pem", "wb") as f:
    f.write(sk.to_pem())
with open("public.pem", "wb") as f:
    f.write(vk.to_pem())

In [3]:
from ecdsa import SigningKey
with open("private.pem") as f:
    sk = SigningKey.from_pem(f.read())
with open("MDReport_V2_26-5387_10452_10331.rep", "rb") as f:
    message = f.read()
sig = sk.sign(message)
with open("MDReport_V2_26-5387_10452_10331.sign", "wb") as f:
    f.write(sig)

In [6]:
import json

def decreaseReporterTrust():
    with open("MDReport_V2_26-5387_10452_10331.rep","r") as fj:
        reportList = json.load(fj)
    reporterId = reportList['Report']['Metadata']['senderRealId']
    print("sebelum:",currReporterTrust[reporterId])
    trust=currReporterTrust[reporterId]
    if(trust>0.66):
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]-0.6)+0.3
    #if(currReporterTrust<0.66 and currReporterTrust>0.32):
    if(trust<0.33):
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]+(1-currReporterTrust[reporterId])*-0.3)
    else:
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]-0.3)+0
    if(trust<0):
        currReporterTrust[reporterId] = 0
    print("setelah:",currReporterTrust[reporterId])
                                         
def increaseReporterTrust():
    with open("MDReport_V2_26-5387_10452_10331.rep","r") as fj:
        reportList = json.load(fj)
    reporterId = reportList['Report']['Metadata']['senderRealId']
    print("sebelum:",currReporterTrust[reporterId])
    trust=currReporterTrust[reporterId]
    if(trust>0.66):
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]+(1-currReporterTrust[reporterId])*0.3)
    if(trust<0.33):
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]-0)+0.3
    #if(currReporterTrust[reporterId]<0.66 and currReporterTrust>=0.33):
    else:
        currReporterTrust[reporterId] = (currReporterTrust[reporterId]-0.3)+0.6

    if(currReporterTrust[reporterId]>1):
        currReporterTrust[reporterId] = 1                                 
    print("setelah:",currReporterTrust[reporterId])


In [7]:
from ecdsa import VerifyingKey, BadSignatureError
vk = VerifyingKey.from_pem(open("public.pem").read())
with open("MDReport_V2_26-5387_10452_10331.rep", "rb") as f:
    message = f.read()
with open("MDReport_V2_26-5387_10452_10331.sign", "rb") as f:
    sig = f.read()
try:
    vk.verify(sig, message)
    print("accepted, lanjutkan proses berikutnya")
    increaseReporterTrust() 
    
except BadSignatureError:
    print("Ditolak, turunkan trust pelapor")
    decreaseReporterTrust()

Ditolak, turunkan trust pelapor
sebelum: 0.28
setelah: 0.06400000000000003
